# PER_FAMILY

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
# Import dataset
location_holidays = "C:\\Users\\75558\\OneDrive - Bain\\Documents\\KAGGLE\\Sales project\\holidays_events.csv"
holidays = pd.read_csv(location_holidays, parse_dates=['date'])
location_oil = "C:\\Users\\75558\\OneDrive - Bain\\Documents\\KAGGLE\\Sales project\\oil.csv"
oil = pd.read_csv(location_oil, parse_dates=['date'])
location_stores = "C:\\Users\\75558\\OneDrive - Bain\\Documents\\KAGGLE\\Sales project\\stores.csv"
stores = pd.read_csv(location_stores)
location_transactions = "C:\\Users\\75558\\OneDrive - Bain\\Documents\\KAGGLE\\Sales project\\transactions.csv"
transactions = pd.read_csv(location_transactions, parse_dates=['date'])
location_train = "C:\\Users\\75558\\OneDrive - Bain\\Documents\\KAGGLE\\Sales project\\train.csv"
train = pd.read_csv(location_train, parse_dates=['date'])
location_test = "C:\\Users\\75558\\OneDrive - Bain\\Documents\\KAGGLE\\Sales project\\test.csv"
test = pd.read_csv(location_test, parse_dates=['date'])

In [37]:
df=train.copy()

In [38]:
#Divide df into train and validation
train_size = int(len(df)*0.8)
df_80, df_20 = df[:train_size], df[train_size:]

In [39]:
#Define df_80_family
df_80_family = df_80.groupby(['date', 'family']).agg({'sales':'sum'}).reset_index()
#Define df_20_family
df_20_family = df_20.groupby(['date', 'family']).agg({'sales':'sum'}).reset_index()
#Print family = 'AUTOMOTIVE'
print(df_80_family[df_80_family['family']=='AUTOMOTIVE'])

            date      family  sales
0     2013-01-01  AUTOMOTIVE    0.0
33    2013-01-02  AUTOMOTIVE  255.0
66    2013-01-03  AUTOMOTIVE  161.0
99    2013-01-04  AUTOMOTIVE  169.0
132   2013-01-05  AUTOMOTIVE  342.0
...          ...         ...    ...
44319 2016-09-08  AUTOMOTIVE  263.0
44352 2016-09-09  AUTOMOTIVE  280.0
44385 2016-09-10  AUTOMOTIVE  371.0
44418 2016-09-11  AUTOMOTIVE  431.0
44451 2016-09-12  AUTOMOTIVE   34.0

[1348 rows x 3 columns]


In [42]:
#Drop family = 'BOOKS'
df_80_family = df_80_family.drop(df_80_family[df_80_family['family']=='BOOKS'].index)
#Drop first rows with 0 sales per family 
for family in df_80_family['family'].unique():
    while df_80_family[(df_80_family['family']==family)].iloc[0]['sales'] == 0:
        df_80_family = df_80_family.drop(df_80_family[(df_80_family['family']==family)].index[0])


In [46]:
#Print size of each family in df_80_family where size < 1348
for family in df_80_family['family'].unique():
    size = df_80_family[df_80_family['family']==family]['date'].size
    if size < 1348:
        print(family, size)

AUTOMOTIVE 1347
HARDWARE 1347
HOME APPLIANCES 1347
SEAFOOD 1347
PRODUCE 1274
CELEBRATION 984
HOME AND KITCHEN I 984
HOME AND KITCHEN II 984
HOME CARE 984
MAGAZINES 984
PET SUPPLIES 984
PLAYERS AND ELECTRONICS 984
LADIESWEAR 983
SCHOOL AND OFFICE SUPPLIES 983
BABY CARE 925


In [ ]:
print(df_20_family[df_20_family['family']=='AUTOMOTIVE']['date'].size)

337


## PROPHET

In [43]:
from prophet import Prophet
for family in df_80_family['family'].unique():
    df_family = df_80_family[df_80_family['family'] == family][['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})
    model = Prophet()
    model.fit(df_family)
    future = model.make_future_dataframe(periods=337)  # Forecasting df_20 size
    forecast = model.predict(future)
    
    #Calculate RMSE
    df_forecast = forecast[['ds', 'yhat']].set_index('ds').join(df_20_family[df_20_family['family'] == family][['date', 'sales']].set_index('date'))
    rmse = np.sqrt(np.mean((df_forecast['yhat'] - df_forecast['sales'])**2))
    print(f'Family: {family}, RMSE: {rmse}')

    # Plot the forecast
    '''fig = model.plot(forecast)
    plt.title(f'Forecast for {family}')
    plt.show()'''

13:18:08 - cmdstanpy - INFO - Chain [1] start processing
13:18:08 - cmdstanpy - INFO - Chain [1] done processing


Family: BEAUTY, RMSE: 87.6724903432936


13:18:08 - cmdstanpy - INFO - Chain [1] start processing
13:18:09 - cmdstanpy - INFO - Chain [1] done processing
13:18:09 - cmdstanpy - INFO - Chain [1] start processing


Family: BEVERAGES, RMSE: 40058.065199947305


13:18:09 - cmdstanpy - INFO - Chain [1] done processing


...

Family: LADIESWEAR, RMSE: 295.9691620180531


13:18:44 - cmdstanpy - INFO - Chain [1] start processing
13:18:45 - cmdstanpy - INFO - Chain [1] done processing


Family: SCHOOL AND OFFICE SUPPLIES, RMSE: 706.7533105900417


13:18:45 - cmdstanpy - INFO - Chain [1] start processing
13:18:45 - cmdstanpy - INFO - Chain [1] done processing


Family: BABY CARE, RMSE: 7.180254451382197


## XGBREGRESSOR

In [27]:
#Train XGBRegressor model for each family
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
for family in df_80_family['family'].unique():
    df_family = df_80_family[df_80_family['family'] == family][['date', 'sales']]
    df_family['month'] = df_family['date'].dt.month
    df_family['day'] = df_family['date'].dt.day
    df_family['year'] = df_family['date'].dt.year
    #Lag Features
    for lag in [1, 7, 14, 30]:
        df_family[f'lag_{lag}'] = df_family['sales'].shift(lag)
    df_family = df_family.dropna()

    #Define X and y
    X = df_family[['month', 'day', 'year', 'lag_1', 'lag_7', 'lag_14', 'lag_30']]
    y = df_family['sales']
    
    #Train model
    model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
    model.fit(X, y)
    
    #Predict on df_20_family
    df_family_20 = df_20_family[df_20_family['family'] == family][['date', 'sales']]
    df_family_20['month'] = df_family_20['date'].dt.month
    df_family_20['day'] = df_family_20['date'].dt.day
    df_family_20['year'] = df_family_20['date'].dt.year
    for lag in [1, 7, 14, 30]:
        df_family_20[f'lag_{lag}'] = df_family['sales'].shift(lag).iloc[-len(df_family_20):].values
    df_family_20 = df_family_20.dropna()
    X_20 = df_family_20[['month', 'day', 'year', 'lag_1', 'lag_7', 'lag_14', 'lag_30']]
    y_20 = df_family_20['sales']
    y_pred = model.predict(X_20)
    rmse = np.sqrt(mean_squared_error(y_20, y_pred))
    print(f'Family: {family}, RMSE: {rmse}')
    
    


Family: AUTOMOTIVE, RMSE: 120.45232045217439
Family: BABY CARE, RMSE: 9.740368757121296
Family: BEAUTY, RMSE: 118.32682986808001
Family: BEVERAGES, RMSE: 58294.828418954276
Family: BOOKS, RMSE: 30.2159389620619
Family: BREAD/BAKERY, RMSE: 6832.135523142801
Family: CELEBRATION, RMSE: 265.16345970058376
Family: CLEANING, RMSE: 17071.667566991928
Family: DAIRY, RMSE: 12859.688166306927
Family: DELI, RMSE: 4393.940249874982
Family: EGGS, RMSE: 2833.945050466571
Family: FROZEN FOODS, RMSE: 17875.17603401565
Family: GROCERY I, RMSE: 71431.87185759058
Family: GROCERY II, RMSE: 550.0281977832957
Family: HARDWARE, RMSE: 24.456356842360638
Family: HOME AND KITCHEN I, RMSE: 755.77640578085
Family: HOME AND KITCHEN II, RMSE: 696.4622587136969
Family: HOME APPLIANCES, RMSE: 21.062728303764512
Family: HOME CARE, RMSE: 4895.877894024594
Family: LADIESWEAR, RMSE: 319.0402375064701
Family: LAWN AND GARDEN, RMSE: 739.1570211533772
Family: LINGERIE, RMSE: 135.1637674098342
Family: LIQUOR,WINE,BEER, RMSE:

## RNN